# Genomic Data Processing Pipeline
This notebook processes genomic VCF data using Hail for machine learning preparation.
## Setup and Requirements
- Hail
- Pandas
- Python 3.9+
## Pipeline Overview
1. Data Loading
2. Quality Control
3. Feature Extraction
## 1. Initial Setup
First, we'll import our required libraries and set up logging utilities.

### Create a temporary directory path

## Initialize Hail for local processing

In [1]:
import hail as hl
import os
import tempfile
import time
from urllib.parse import urlparse, urlunparse

def log_step(message):
    """Utility function to log steps with timestamps"""
    print(f"\n[{time.strftime('%H:%M:%S')}] {message}")

def _get_local_tmpdir():
    tmp_path = tempfile.gettempdir()
    if os.name == 'nt':  # Windows
        tmp_url = f'file:///{tmp_path.replace(os.sep, "/")}'
    else:  # Unix-like systems
        tmp_url = f'file://{tmp_path}'
    r = urlparse(tmp_url)
    if not r.scheme:
        r = r._replace(scheme='file')
    return urlunparse(r)

try:
    # Check if Hail is already initialized
    if hl.eval(hl.literal(1)) == 1:
        print("Hail is already initialized")

except Exception:
    # If Hail is not initialized or there's an error, initialize it
    local_tmpdir = _get_local_tmpdir()
    print(f"Initializing Hail with temporary directory: {local_tmpdir}")
    
    hl.init(
        default_reference="GRCh38",
        local_tmpdir=local_tmpdir
    )
    print("Hail initialized successfully")

Loading BokehJS ...

Initializing Hail with default parameters...
Running on Apache Spark version 3.5.4
SparkUI available at http://LT2058-20241028.SHTC.com:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.133-4c60fddb171a
LOGGING: writing to c:\Users\jmcdonald\source\PersonalRepos\genome-analysis-ml\notebooks\hail-20250112-1231-0.2.133-4c60fddb171a.log


Hail is already initialized


In [3]:
# Configure your VCF file path
vcf_path = r'C:\Users\jmcdonald\Desktop\Projects\The Genome Project\gnomad.exomes.v4.1.sites.chr13.vcf.bgz'
print(f"Checking if file exists: {os.path.exists(vcf_path)}")
log_step("Loading VCF file...")
try:
    mt = hl.import_vcf(vcf_path)
    
    # Try getting the counts in separate try-except blocks
    try:
        n_variants = mt.count_rows()
        print(f"Number of variants: {n_variants:,}")
    except Exception as e:
        print(f"Error counting variants: {e}")
    
    try:
        n_samples = mt.count_cols()
        print(f"Number of samples: {n_samples:,}")
    except Exception as e:
        print(f"Error counting samples: {e}")
        
except Exception as e:
    print(f"Error loading VCF: {e}")

Checking if file exists: True

[12:32:40] Loading VCF file...
Error counting variants: HailException: Invalid locus 'chr13:48379904' found. Contig 'chr13' is not in the reference genome 'GRCh37'.

Java stack trace:
org.apache.spark.SparkException: Job aborted due to stage failure: Task 12 in stage 1.0 failed 1 times, most recent failure: Lost task 12.0 in stage 1.0 (TID 38) (LT2058-20241028.SHTC.com executor driver): is.hail.utils.HailException: file:/C:/Users/jmcdonald/Desktop/Projects/The Genome Project/gnomad.exomes.v4.1.sites.chr13.vcf.bgz:offset 107083150656189: error while parsing line
chr13	48379904	.	G	A	.	AC0	AC=0;AN=614612;AF=0.00000;AC_XX=0;AF_XX=0.00000;AN_XX=297944;nhomalt_XX=0;AC_XY=0;AF_XY=0.00000;AN_XY=316668;nhomalt_XY=0;nhomalt=0;AC_afr_XX=0;AF_afr_XX=0.00000;AN_afr_XX=8146;nhomalt_afr_XX=0;AC_afr_XY=0;AF_afr_XY=0.00000;AN_afr_XY=6494;nhomalt_afr_XY=0;AC_afr=0;AF_afr=0.00000;AN_afr=14640;nhomalt_afr=0;AC_amr_XX=0;AF_amr_XX=0.00000;AN_amr_XX=7498;nhomalt_amr_XX=0;AC_am

In [8]:
print(f"Initial dataset dimensions:")
print(f"- Number of variants: {mt.count_rows():,}")
print(f"- Number of samples: {mt.count_cols():,}")

Initial dataset dimensions:


FatalError: HailException: Invalid locus 'chr13:108215366' found. Contig 'chr13' is not in the reference genome 'GRCh37'.

Java stack trace:
org.apache.spark.SparkException: Job aborted due to stage failure: Task 22 in stage 0.0 failed 1 times, most recent failure: Lost task 22.0 in stage 0.0 (TID 22) (LT2058-20241028.SHTC.com executor driver): is.hail.utils.HailException: file:/C:/Users/jmcdonald/Desktop/Projects/The Genome Project/gnomad.exomes.v4.1.sites.chr13.vcf.bgz:offset 196318482477539: error while parsing line
chr13	108215366	.	G	T	.	AC0	AC=0;AN=334;AF=0.00000;AC_XX=0;AF_XX=0.00000;AN_XX=88;nhomalt_XX=0;AC_XY=0;AF_XY=0.00000;AN_XY=246;nhomalt_XY=0;nhomalt=0;AC_afr_XX=0;AF_afr_XX=0.00000;AN_afr_XX=4;nhomalt_afr_XX=0;AC_afr_XY=0;AF_afr_XY=0.00000;AN_afr_XY=6;nhomalt_afr_XY=0;AC_afr=0;AF_afr=0.00000;AN_afr=10;nhomalt_afr=0;AC_amr_XX=0;AN_amr_XX=0;nhomalt_amr_XX=0;AC_amr_XY=0;AF_amr_XY=0.00000;AN_amr_XY=2;nhomalt_amr_XY=0;AC_amr=0;AF_amr=0.00000;AN_amr=2;nhomalt_amr=0;AC_asj_XX=0;